In [34]:
import escher
from escher import Builder
import cobra
from time import sleep

In [35]:
# Load the model into cobra
model = cobra.io.load_json_model('e_coli_core.json')

In [36]:
# Make a medium with only glucose and limited oxygen
glc_medium = {
    'EX_co2_e': 1000.0,
    'EX_glc__D_e': 10.0,
    'EX_h_e': 1000.0,
    'EX_h2o_e': 1000.0,
    'EX_nh4_e': 1000.0,
    'EX_o2_e': 20.0,
    'EX_pi_e': 1000.0
}

In [37]:
# Run FBA for the default medium
model.medium = glc_medium
glc_10_solution = model.optimize()

In [46]:
# Make a map with the solution for the default medium
glc_10_map = Builder(
    map_name = 'e_coli_core.Core metabolism',
    reaction_data = glc_10_solution.fluxes
)

In [47]:
# View the map
glc_10_map

Builder(reaction_data={'PFK': 7.425175614551203, 'PFL': 0.0, 'PGI': 4.589443418031537, 'PGK': -16.040039558585…

In [40]:
# Define medium with just a little bit of acetate available
# Note, this does not set the flux, just the limits on it
glc_ace_medium = {
    'EX_co2_e': 1000.0,
    'EX_glc__D_e': 10.0,
    'EX_ac_e': 1,
    'EX_h_e': 1000.0,
    'EX_h2o_e': 1000.0,
    'EX_nh4_e': 1000.0,
    'EX_o2_e': 20.0,
    'EX_pi_e': 1000.0
}

In [41]:
# Run FBA for the glucose + acetate medium
model.medium = glc_ace_medium
ace_1_solution = model.optimize()

In [42]:
# Make a map with the solution for the defulat medium
ace_1_map = Builder(
    map_name = 'e_coli_core.Core metabolism',
    reaction_data = ace_1_solution.fluxes
)

In [43]:
ace_1_map

Builder(reaction_data={'PFK': 7.425175614551202, 'PFL': 0.0, 'PGI': 4.589443418031526, 'PGK': -16.040039558585…

In [44]:
# Get a list of the reactions that are different between the two solutions, with the difference between them
diff_reactions = {}
for reaction in model.reactions:
    if abs(glc_10_solution.fluxes[reaction.id] - ace_1_solution.fluxes[reaction.id]) > 1e-6:
        diff_reactions[reaction.id] = ace_1_solution.fluxes[reaction.id] - glc_10_solution.fluxes[reaction.id]

In [45]:
# Print the reaction ID, the reaction name, the flux in the glucose medium, and the flux in the glucose + acetate medium in the order of the difference between the two
for reaction in sorted(diff_reactions, key = lambda x: abs(diff_reactions[x])):
    print(reaction, model.reactions.get_by_id(reaction).name, glc_10_solution.fluxes[reaction], ace_1_solution.fluxes[reaction])

In [48]:
model.reactions.ACt2r

Reaction identifier,ACt2r
Name,Acetate reversible transport via proton symport
Memory address,0x7f8256434310
Stoichiometry,ac_e + h_e <=> ac_c + h_c Acetate + H+ <=> Acetate + H+
GPR,
Lower bound,-1000.0
Upper bound,1000.0


In [49]:
model.reactions.ACKr

Reaction identifier,ACKr
Name,Acetate kinase
Memory address,0x7f820583c460
Stoichiometry,ac_c + atp_c <=> actp_c + adp_c Acetate + ATP C10H12N5O13P3 <=> Acetyl phosphate + ADP C10H12N5O10P2
GPR,b3115 or b2296 or b1849
Lower bound,-1000.0
Upper bound,1000.0
